### Домашнее задание №3 (Оцениваемое)

#### Задание №1


Скачайте текст "Литературных анекдотов". Напишите функцию, которая будет читать файл, лемматизировать текст с помощью pymystem3 и записывать результат в новый файл. У функции должно бы два аргумента: путь к исходному файлу и путь к файлу с лемматизированным текстом. Вызов функции тоже должен быть прописан в решении.
    

In [1]:
from pymystem3 import Mystem
from string import punctuation
from collections import Counter
import json
import re


In [2]:
read_path = 'C:\\Users\\Нелли\\Documents\\python-for-dh\\Classes\\9-10\\literary_anecdotes.txt'
write_path = 'C:\\Users\\Нелли\\Documents\\python-for-dh\\Classes\\9-10\\lemmatized_anecdotes.json'
stopwords_path = 'C:\\Users\\Нелли\\Documents\\python-for-dh\\Classes\\rus_stopwords.txt'

In [3]:
def text_lemmatization(original_text): #создаем функцию для лемматизации текста
    m = Mystem()
    new_text = [w.strip(punctuation) for w in original_text.split()] #очищаем текст от пунктуации
    united_line = ' '.join(new_text) # преобразуем список в строку
    lemma = m.lemmatize(united_line) #создаем переменную, в которую будет записан лемматизованный текст
    return ''.join(lemma) #возвращаем лематизированный текст

In [4]:
def text_preprocessing(read_path, write_path): #создаем функцию для чтениня и записи нового результата в файл
    f = open(read_path, 'r', encoding='utf-8') #считываем текст из файла
    text = f.read()
    lemmatized_text = text_lemmatization(text) #записываем в переменную результата вызова функции лемматизации
    with open(write_path, 'w', encoding='utf-8') as f: #открываем файл
        json.dump(lemmatized_text, f, ensure_ascii=False) #записываем в файл новый результат

In [5]:
text_preprocessing(read_path, write_path) #вызываем созданную раннее функцию

#### Задание №2

Очистите лемматизированный текст от стоп-слов и посчитайте ipm для оставшихся. Выведите 20 самых частотных по этому параметру слов.

In [6]:
def delete_stopwords(original_text, stopwords): #создадим функцию для удаления стоп-слов
    new_stopwordless_text = [w.strip(punctuation) for w in original_text if w not in stopwords] #если слово не входит в список в стоп-слов, оставляем его и добавляем в массив
    return new_stopwordless_text #возвращаем текст без стоп-слов

In [7]:
with open(write_path, 'r', encoding='utf-8') as f:
        lemmatized_text = f.read().split() #открываем файл с лемматизированным текстом, считываем его в переменную, преобразуя в список
        
with open(stopwords_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().split('\n') #открываем файл со стоп-словами, считываем его в переменную, преобразуя в список
punctuation += "« » — \n ''"
new_stopwordless_text = delete_stopwords(lemmatized_text, stopwords) #записываем в переменную очищенный от стоп-слов текст

In [8]:
words_frequency = Counter(new_stopwordless_text) #создаем словарь, где ключом будет слово, а значением - его частотность
total_words = (len(lemmatized_text)) #считаем абсолютную частотность
for key, value in words_frequency.items(): #проходимся циклом по словарю и обновляем его значение на ipm
    words_frequency[key] = value/total_words * 1000000 #обновление значений

In [9]:
print('Самые частотные слова: ', words_frequency.most_common(20)) #получаем список 20 самых частотных слов

Самые частотные слова:  [('пушкин', 18987.3417721519), ('толстой', 11251.758087201126), ('гоголь', 10900.14064697609), ('однажды', 10196.905766526019), ('лев', 8790.43600562588), ('любить', 7032.3488045007025), ('достоевский', 6680.731364275668), ('тургенев', 5274.261603375528), ('ребенок', 4922.644163150492), ('царствие', 4571.026722925457), ('окно', 4219.4092827004215), ('бульвар', 4219.4092827004215), ('идти', 4219.4092827004215), ('лермонтов', 4219.4092827004215), ('приходить', 3867.791842475387), ('тверской', 3867.791842475387), ('федор', 3867.791842475387), ('михайлович', 3867.791842475387), ('герцен', 3516.1744022503512), ('небесный', 3516.1744022503512)]


#### Задание №3

Сделайте полный морфологический разбор исходного текста. Напишите регулярное выражение, которое будет извлекать из тега только часть речи. Пройдитесь циклом по списку с разборами, который выдал pymystem3, извлекая из каждого разбора форму слова и его часть речи и записывая их в новый словарь (форма -- ключ, часть речи -- значение). Посчитайте абсолютную частоту для всех частей речи, а затем относительнную частоту (абсолютная частота / длина текста в словах).

In [10]:
m = Mystem()
f = open(read_path, 'r', encoding='utf-8')
literary_anecdotes = f.read() #читаем файл с исходным текстом
literary_anecdotes = ' '.join([w.strip(punctuation) for w in literary_anecdotes.split()]) #избавляемся от пунктуации
word_analysis = m.analyze(literary_anecdotes) #проводим морфологический разбор текста и записываем результат в переменную

form_pos = {} #создаем пустой словарь
pos_reg = re.compile("[A-Z]+") #создаем регулярное выражение для нахождени части речи (части речи отображаются в виде аббревиатур английскими буквами)
for word in word_analysis: #проитерируемся по массиву с результатами анализа
    try:
        if len(word['analysis'])>0: #исключим варианты, где выдается пустой анализ
            gr = word['analysis'][0]['gr'] #выделим грамматический разбор
            pos = pos_reg.match(gr).group(0) #выделяем часть речи
            form = word['text'] #выделим форму слова
            if form != '': #исключим варианты, когда у нам появляется пустой символ
                form_pos[form] = pos #записываем результаты в словарь, преобразовывая массив в строку
    except (KeyError, IndexError): #обрабатываем исключения
        pass

In [11]:
pos_frequency = Counter(form_pos.values()) #абсолютная частота частей речи
relative_pos_frequency = dict() #создадим пустой словарь, куда запишем значения относительной частоты
number_of_words_total = len(literary_anecdotes.split()) #найдем длину текста в словах
for key, value in pos_frequency.items(): #пройдемся циклом по значениям абсолютной частоты частей речи
    relative_pos_frequency[key] = value/number_of_words_total #получим относительную частоту частей речи

In [12]:
print('Абсолютная частота частей речи: \n', pos_frequency)
print('Относительная частота частей речи: \n', relative_pos_frequency)

Абсолютная частота частей речи: 
 Counter({'S': 507, 'V': 426, 'A': 98, 'ADV': 89, 'SPRO': 54, 'APRO': 49, 'PR': 37, 'PART': 31, 'ADVPRO': 24, 'CONJ': 20, 'NUM': 9, 'INTJ': 8, 'ANUM': 3})
Относительная частота частей речи: 
 {'PR': 0.013009845288326301, 'S': 0.17827004219409281, 'V': 0.14978902953586498, 'A': 0.034458509142053444, 'ADV': 0.03129395218002813, 'SPRO': 0.0189873417721519, 'CONJ': 0.007032348804500703, 'PART': 0.01090014064697609, 'ADVPRO': 0.008438818565400843, 'APRO': 0.017229254571026722, 'ANUM': 0.0010548523206751054, 'INTJ': 0.0028129395218002813, 'NUM': 0.0031645569620253164}


Если использовать ipm:

In [13]:
relative_pos_frequency_ipm = dict() #создадим пустой словарь, куда запишем значения относительной частоты
for key, value in pos_frequency.items(): #пройдемся циклом по значениям абсолютной частоты частей речи
    relative_pos_frequency_ipm[key] = value/number_of_words_total * 1000000 #получим относительную частоту частей речи (используя ipm)

In [14]:
print('Относительная частота частей речи (используя ipm): \n', relative_pos_frequency_ipm)

Относительная частота частей речи (используя ipm): 
 {'PR': 13009.845288326302, 'S': 178270.04219409282, 'V': 149789.02953586497, 'A': 34458.509142053445, 'ADV': 31293.952180028133, 'SPRO': 18987.3417721519, 'CONJ': 7032.3488045007025, 'PART': 10900.14064697609, 'ADVPRO': 8438.818565400843, 'APRO': 17229.254571026722, 'ANUM': 1054.8523206751054, 'INTJ': 2812.9395218002815, 'NUM': 3164.5569620253164}
